In [1]:
import pandas as pd

df = pd.read_csv('data/coordenadas/zonificacion_coordenadas_cafe.csv')
df.head()

,id,cod_depart,departamen,cod_dane_m,municipio,lon,lat,area_ha,aptitud
0,1,5,Antioquia,5001,Medellín,-75.611288,6.258531,1133.151245,Aptitud alta
1,2,5,Antioquia,5002,Abejorral,-75.438474,5.804174,13323.429688,No apta
2,3,5,Antioquia,5004,Abriaquí,-76.085735,6.626307,29114.949219,Exclusión legal
3,4,5,Antioquia,5021,Alejandría,-75.090177,6.364923,2455.476562,No apta
4,5,5,Antioquia,5030,Amagá,-75.707902,6.032773,355.028381,Exclusión legal


In [ ]:
import requests
import pandas as pd
from pathlib import Path
import time, random
from tqdm import tqdm

# Configuraciones generales
COORDS_CSV = "data/coordenadas/zonificacion_coordenadas_cafe.csv"
OUT_DIR = Path("data/nasa_power")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Departamentos a conservar
KEEP_DEPARTS = {63, 25}  # Quindío y Cundinamarca

# Rango de años a consultar
START_YEAR = 2014
END_YEAR   = 2025  # solo hasta julio 2025

# Configuración API NASA POWER
NASA_URL = "https://power.larc.nasa.gov/api/temporal/daily/point"

PARAMETERS = ",".join([
    "T2M","RH2M","ALLSKY_SFC_SW_DWN","PRECTOTCORR",
    "CLOUD_AMT","WS2M","GWETPROF","V50M","TS_ADJ","TQV"
])

API_CONFIG = {
    "community": "AG",
    "format": "JSON",
    "units": "metric",
    "time-standard": "UTC"
}

# Funciones 
def load_coords():
    df = pd.read_csv(COORDS_CSV)
    df.columns = [c.lower() for c in df.columns]
    return df[df["cod_depart"].isin(KEEP_DEPARTS)]

def call_nasa(lat, lon, start, end):
    params = {
        "start": start,
        "end": end,
        "latitude": lat,
        "longitude": lon,
        "parameters": PARAMETERS,
        **API_CONFIG
    }
    r = requests.get(NASA_URL, params=params, timeout=120)
    r.raise_for_status()
    return r.json()

def parse_json(data):
    times = list(data["properties"]["parameter"]["T2M"].keys())
    rows = []
    for t in times:
        row = {"date": t}
        for var in PARAMETERS.split(","):
            row[var] = data["properties"]["parameter"][var].get(t)
        rows.append(row)
    df = pd.DataFrame(rows)
    df["date"] = pd.to_datetime(df["date"])
    return df


# Proceso principal
df_coords = load_coords()

for _, row in tqdm(df_coords.iterrows(), total=len(df_coords)):
    uid, lat, lon = row["id"], row["lat"], row["lon"]
    out = OUT_DIR / f"{uid}_nasa_power.csv"
    
    if out.exists():
        print("ya existe:", uid)
        continue

    full_df = []
    print(f"\nID {uid} ({lat},{lon})")

    # Consulta por año
    for year in range(START_YEAR, END_YEAR + 1):

        # Validar rangos de fechas que hay datos almacenados
        
        if year == 2025:
            start, end = "20250101", "20250731"  # hasta julio 31 de 2025
        else:
            start, end = f"{year}0101", f"{year}1231"

        try:
            print(f"   → {year} ({start} a {end})")
            j = call_nasa(lat, lon, start, end)
            df_year = parse_json(j)
            full_df.append(df_year)
            time.sleep(random.uniform(2, 3))
        except Exception as e:
            print("Error:", e)

    if full_df:
        final = pd.concat(full_df)
        final.to_csv(out, index=False)
        print("Guardado:", out)

  0%|          | 0/129 [00:00<?, ?it/s]


📍 ID 149 (4.316707855240076,-74.18107344315469)
   → 2014
   → 2015
   → 2016
   → 2017
   → 2018
   → 2019
   → 2020
   → 2021
   → 2022
   → 2023
   → 2024
   → 2025


  1%|          | 1/129 [00:57<2:03:38, 57.95s/it]

✅ Guardado: data/nasa_power/149_nasa_power.csv

📍 ID 459 (4.372601518177892,-74.67078936139487)
   → 2014
   → 2015
   → 2016
   → 2017
   → 2018
   → 2019
   → 2020
   → 2021
   → 2022
   → 2023
   → 2024
   → 2025


  2%|▏         | 2/129 [01:53<1:59:10, 56.30s/it]

✅ Guardado: data/nasa_power/459_nasa_power.csv

📍 ID 460 (4.891364173619755,-74.4428272028283)
   → 2014
   → 2015
   → 2016
   → 2017
   → 2018
   → 2019
   → 2020
   → 2021
   → 2022
   → 2023
   → 2024
   → 2025


  2%|▏         | 3/129 [02:50<1:59:02, 56.69s/it]

✅ Guardado: data/nasa_power/460_nasa_power.csv

📍 ID 461 (4.559934233644917,-74.52535182004121)
   → 2014
   → 2015
   → 2016
   → 2017
   → 2018
   → 2019
   → 2020
   → 2021


  2%|▏         | 3/129 [03:25<2:23:42, 68.43s/it]


KeyboardInterrupt: 